In [1]:
! pip install beautifulsoup4
! pip install requests
! pip install pandas
! pip install mysql-connector-python

In [3]:
import requests
import mysql.connector
from bs4 import BeautifulSoup
import uuid 
import time
import datetime
from dateutil.parser import parse

In [7]:
songplay_table_drop = "DROP TABLE IF EXISTS songplays;"
user_table_drop = "DROP TABLE IF EXISTS users;"
song_table_drop = "DROP TABLE IF EXISTS songs;"
artist_table_drop = "DROP TABLE IF EXISTS artists;"
time_table_drop = "DROP TABLE IF EXISTS time;"

# CREATE TABLES

songplay_table_create = ("""create table if not exists songplays(songplay_id SERIAL PRIMARY KEY NOT NULL, start_time timestamp,user_id int,level varchar, song_id varchar, artist_id varchar,session_id int,location varchar,user_agent varchar);""")

user_table_create = ("""create table if not exists users(user_id int PRIMARY KEY NOT NULL,first_name varchar,last_name varchar,gender varchar, level varchar);""")

song_table_create = ("""create table if not exists songs(song_id varchar PRIMARY KEY NOT NULL,title varchar,artist_id varchar,year int,duration numeric);""")

artist_table_create = ("""create table if not exists artists(artist_id varchar PRIMARY KEY NOT NULL, name varchar, location varchar, latitude numeric, longitude numeric);""")

time_table_create = ("""create table if not exists time(start_time timestamp PRIMARY KEY NOT NULL, hour int, day int, week int, month int, year int, weekday int);""")


create_table_queries = [songplay_table_create, user_table_create, song_table_create, artist_table_create, time_table_create]
drop_table_queries = [songplay_table_drop, user_table_drop, song_table_drop, artist_table_drop, time_table_drop]


In [6]:
import mysql.connector


def create_database():
    conn = mysql.connector.connect(
      host="localhost",
      user="root",
      password=""
    )
    cur = conn.cursor()

    cur.execute("CREATE DATABASE stock_performance_1")
    
    return cur, conn

def drop_tables(cur, conn):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()


def create_tables(cur, conn):
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()


def main():
    cur, conn = create_database()
    print("Created database ")
    
    drop_tables(cur, conn)
    create_tables(cur, conn)

    conn.close()


if __name__ == "__main__":
    main()

In [ ]:


main_url = 'https://hamptonroadsalliance.com/'
try:
    page = requests.get(main_url, timeout=5)
    sector_soup = BeautifulSoup(page.content, 'html.parser')
    main_page_content = sector_soup.find(id = 'page#26')
except:
    print("Could not scrape for some reason")

# Get Sector Names
sectors_list = ()
for a in main_page_content.find_all('a', href=True): 
    if a.text: 
        value = a['href']
        sectors_list=sectors_list+(value.replace('/',''),)

# Get Sectors list
sectors_list_array = []
for value in sectors_list:
    sectors_list_array.append((str(uuid.uuid4()), value))
print(sectors_list_array)

In [ ]:
# Database Connection

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="stock_performance"
)
mycursor = mydb.cursor()

In [ ]:
 # Insert data into sector table   
sql = "INSERT INTO sector (sector_id, sector_name) VALUES (%s, %s)"
mycursor.executemany(sql,sectors_list_array)
mydb.commit()
print(mycursor.rowcount, "record inserted")

In [ ]:
# Get companies list as tuple
for sector in sectors_list:
    try:
        page = requests.get(main_url+sector,timeout=5)
        company_soup = BeautifulSoup(page.content, 'html.parser')
    except:
        print("Could not scrape for some reason")
    companies_list = ()
    companies_list = companies_list + ()
    for a in company_soup.find_all("h4","el-title uk-h4 uk-margin-top uk-margin-remove-bottom"):
        if a.text:
            value = a.text.strip()


In [ ]:
# Get companies_list_array as group of companies
# Get sector_companies_list_array
import uuid
sector_companies_list =()
sector_companies_list_array = []
companies_list = ()
companies_list_array = []
for sector in sectors_list:
    sql = "SELECT sector_id FROM sector where sector_name=%s"
    adr = (sector,)
    mycursor.execute(sql, adr)
    sector_id = mycursor.fetchone()[0]
    try:
        page = requests.get(main_url+sector, timeout=5)
        soup = BeautifulSoup(page.content, 'html.parser')
    except:
        print("Could not scrape for some reason")
    sector_companies_list = ()
    sector_companies_list = sector_companies_list + (sector_id,)
    for a in soup.find_all("h4","el-title uk-h4 uk-margin-top uk-margin-remove-bottom"):
        if a.text:
            value = a.text.strip()
            uuid_value = str(uuid.uuid4())
            companies_list=companies_list+(uuid_value,value)+ ('',)
            sector_companies_list = sector_companies_list + (uuid_value,)
            sector_companies_list_array.append(sector_companies_list)
            sector_companies_list = ()
            sector_companies_list = sector_companies_list + (sector_id,)
            companies_list_array.append(companies_list)
            companies_list = ()

print (len(companies_list_array))
print (len(sector_companies_list_array))



In [ ]:
# Insert into company table 
insert_into_company_table = "INSERT INTO company (id, name, stock_symbol) VALUES (%s, %s, %s)"
mycursor.executemany(insert_into_company_table,companies_list_array)
mydb.commit()
print(mycursor.rowcount, "record inserted")

In [ ]:
# Insert into and company_sector table
insert_into_sector_company_table = "INSERT INTO company_sector (sector_id, company_id) VALUES (%s, %s)"
mycursor.executemany(insert_into_sector_company_table,sector_companies_list_array) 
mydb.commit()
print(mycursor.rowcount, "record inserted")

In [ ]:


mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="stock_performance"
)
mycursor = mydb.cursor()

# Get companies with not null stock_symbol

sql = "SELECT id,stock_symbol FROM company where stock_symbol is not null"
mycursor.execute(sql)
company_data = mycursor.fetchall()

company_data

In [ ]:
## Construct data for inserting into stock_price_history table
import uuid

sql = "SELECT id FROM stock_source where source_name='Yahoo Finance'"
mycursor.execute(sql)
price_source_id = mycursor.fetchone()[0]
print(price_source_id)


stock_history_array = []
stock_history = ()
for i in company_data:
    ## Get Market
    try:
        page = requests.get('https://finance.yahoo.com/quote/'+i[1], timeout=5)
        stock_market_soup = BeautifulSoup(page.text, 'html.parser')
        market = stock_market_soup.find('div',{'id':'quote-header-info'}).find('span').text
    except:
        print("Could not scrape for some reason")
    print(i[1],market)
    time.sleep(1)
    
    ## Get Stock_history 
    try:
        page = requests.get('https://finance.yahoo.com/quote/'+i[1]+'/history', timeout=5)
        stock_history_soup = BeautifulSoup(page.text, 'html.parser')
    except:
        print("Could not scrape for some reason")
    count = 0
#     stock_history = (uuid_value,i[0],i[1],market,)
    for item in stock_history_soup.find('table',{'data-test':'historical-prices'}).find('tbody'):
        uuid_value = str(uuid.uuid4())
        stock_history = (uuid_value,i[0],i[1],market,price_source_id,)
        if(count < 10):
            count = count + 1
            for value in item.find_all('span'):
                stock_history = stock_history + (value.text,)
            stock_history_array.append(stock_history)
    time.sleep(1)
# print(stock_history_array)


In [ ]:

## Data cleaning to inserting empty string if volume doesn't exist
for i in range(len(stock_history_array)):
    print(len(stock_history_array[i]))
    if(len(stock_history_array[i]) < 11):
        stock_history_array[i] = stock_history_array[i] +('',)
    
## Format date correctly and remove ',' from volume
for i in range(len(stock_history_array)):
    stock_history_array_list = list(stock_history_array[i])
    stock_history_array_list[4] = datetime.datetime.strptime(stock_history_array_list[4], '%b %d, %Y')
    stock_history_array_list[10] = stock_history_array_list[10].replace(",", "")
    stock_history_array[i] = tuple(stock_history_array_list)
print(stock_history_array)

In [ ]:
## Insert into stock_price_history table

insert_into_stock_price_history_table = "INSERT INTO stock_price_history (id, company_id, stock_symbol, market,price_source_id, Date, open_price, high_price, low_price, close_price, adj_close_price, volume) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
mycursor.executemany(insert_into_stock_price_history_table,stock_history_array) 
mydb.commit()
print(mycursor.rowcount, "record inserted")


In [ ]:
### Seeking Alpha - Get Stock Market News Data
import uuid
import time

# Get Scraping source id 

sql = "SELECT id FROM stock_source where source_name='Seeking Alpha Feeds'"
mycursor.execute(sql)
news_source_id = mycursor.fetchone()[0]
print(news_source_id)

## Scrape data to insert into stock_news table

seeking_alpha_news_items_list = []
for i in company_data:
    try:
        sa_stock_market_news_page = requests.get('https://seekingalpha.com/api/sa/combined/'+i[1]+'.xml', timeout=5)
        sa_stock_market_news_soup = BeautifulSoup(sa_stock_market_news_page.content,features="xml")
        sa_items = sa_stock_market_news_soup.findAll('item')
    except:
        print("Could not scrape for some reason")
    for item in sa_items:
        seeking_alpha_news_items = ()
        date = parse(item.pubDate.text).date()
        uuid_value = str(uuid.uuid4())
        seeking_alpha_news_items = seeking_alpha_news_items + (uuid_value,i[0],i[1],item.title.text,item.link.text,date,item.author_name.text,news_source_id,)
        seeking_alpha_news_items_list.append(seeking_alpha_news_items)
        print(seeking_alpha_news_items)
    time.sleep(1)
print(len(seeking_alpha_news_items_list))
    

In [ ]:
## Insert into stock_news table

insert_into_stock_news_table = "INSERT INTO stock_news (id, company_id, stock_symbol, news_title, news_url, published_date, author, news_source_id ) VALUES (%s,%s,%s,%s,%s,%s,%s,%s)"
mycursor.executemany(insert_into_stock_news_table,seeking_alpha_news_items_list) 
mydb.commit()
print(mycursor.rowcount, "record inserted")


In [ ]:
import uuid
import time
## Get news from Yahoo Finance 

# Get Scraping source id 

sql = "SELECT id FROM stock_source where source_name='Yahoo Finance'"
mycursor.execute(sql)
news_source_id = mycursor.fetchone()[0]
# print(news_source_id)

yahoo_finance_news_item_list = []
for i in company_data:
    list_of_news_urls = []
    yahoo_news_page = requests.get('https://finance.yahoo.com/quote/'+i[1],timeout=5)
    yahoo_news_soup = BeautifulSoup(yahoo_news_page.text, 'html.parser')
    yahoo_news_data = yahoo_news_soup.find_all('div',{'id':'quoteNewsStream-0-Stream'})
    print("New Company")
    # Get only 5 News urls data per one company 
    count = 0
    for item in yahoo_news_data:
        a_tags = item.find_all('a')
        for a_tag_item in a_tags:
            yahoo_finance_news_item = ()
            uuid_value = str(uuid.uuid4())
            if a_tag_item.has_attr('href'):
                count= count+1
                news_url = 'https://finance.yahoo.com' + a_tag_item.attrs['href']
                
                ## Scrape single news page for published_date, title and author information
                try:
                    yahoo_single_news_page = requests.get(news_url,timeout=5)
                    yahoo_single_news_page.raise_for_status()
                    yahoo_single_news_soup = BeautifulSoup(yahoo_single_news_page.text, 'html.parser')
                    yahoo_single_news_data = yahoo_single_news_soup.find('div',{'id':'YDC-Side-StackCompositeSideTop'})
                    news_title = yahoo_single_news_data.find('h1',{'class':'Lh(36px) Fz(25px)--sm Fz(32px) Mb(17px)--sm Mb(20px) Mb(30px)--lg Ff($ff-primary) Lts($lspacing-md) Fw($fweight) Fsm($fsmoothing) Fsmw($fsmoothing) Fsmm($fsmoothing) Wow(bw)'}).text
                    author = yahoo_single_news_data.find('div',{'class':'auth-prov-soc Mend(4px) Va(m) D(tbc) Mah(45px) Mah(40px)--sm Maw(320px) Fz(14px)'}).find('a').text
                    published_date = parse(yahoo_single_news_data.find('time').text).date()
                except:
                    print("Some problem")
                
                # Creating a single news record
                yahoo_finance_news_item = yahoo_finance_news_item + (uuid_value,i[0],i[1],news_title,news_url,published_date,author,news_source_id,)
                # Adding each record into a list 
                yahoo_finance_news_item_list.append(yahoo_finance_news_item)
                print(yahoo_finance_news_item)
                if(count==5):
                    break
            time.sleep(1)


In [ ]:
## Insert into stock_news table

insert_into_stock_news_table = "INSERT INTO stock_news (id, company_id, stock_symbol, news_title, news_url, published_date, author, news_source_id ) VALUES (%s,%s,%s,%s,%s,%s,%s,%s)"
mycursor.executemany(insert_into_stock_news_table,yahoo_finance_news_item_list) 
mydb.commit()
print(mycursor.rowcount, "record inserted")
